In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from google.colab.patches import cv2_imshow

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Handwritten_text_converter/data.csv')
df.head()

,path,text
0,r08/r08-580/r08-580z-01.tif,"They appeared to look elsewhere,"
1,r08/r08-580/r08-580z-02.tif,"indeed, until I realized afresh, as"
2,r08/r08-580/r08-580z-03.tif,"you have to, that all birds look"
3,r08/r08-580/r08-580z-04.tif,at you from the sides of their
4,r08/r08-580/r08-580z-05.tif,heads.


In [ ]:
path = '/content/drive/MyDrive/Handwritten_text_converter/'
path_img = path + 'lineImages'

In [ ]:
import sys        
sys.path.append('/content/drive/MyDrive/Handwritten_text_converter/Word_detector')  

In [ ]:
def prepare_image(img, height):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  h = gray.shape[0]
  factor = height/h
  return cv2.resize(gray, dsize = None, fx = factor, fy = factor)

def prepare_kernel(kernel_size, sigma, theta):
  half_size = kernel_size // 2
  xs = ys = np.linspace(-half_size, half_size, kernel_size)
  x, y = np.meshgrid(xs, ys)

  # compute sigma values in x and y direction, where theta is roughly the average x/y ratio of words
  sigma_y = sigma
  sigma_x = sigma_y * theta

  # compute terms and combine them
  exp_term = np.exp(-x ** 2 / (2 * sigma_x) - y ** 2 / (2 * sigma_y))
  x_term = (x ** 2 - sigma_x ** 2) / (2 * np.math.pi * sigma_x ** 5 * sigma_y)
  y_term = (y ** 2 - sigma_y ** 2) / (2 * np.math.pi * sigma_y ** 5 * sigma_x)
  kernel = (x_term + y_term) * exp_term

  # normalize and return kernel
  kernel = kernel / np.sum(kernel)
  return kernel

def detect(img, min_area, kernel_size, sigma, theta):
  kernel = prepare_kernel(kernel_size, sigma, theta)
  img_filtered = cv2.filter2D(img, -1, kernel, borderType=cv2.BORDER_REPLICATE).astype(np.uint8)
  img_thres = 255 - cv2.threshold(img_filtered, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
  res = []
  components = cv2.findContours(img_thres, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[0]
  for c in components:
    # skip small word candidates
    if cv2.contourArea(c) < min_area:
        continue
    # append bounding box and image of word to result list
    x, y, w, h = cv2.boundingRect(c)  # bounding box as tuple (x, y, w, h)
    crop = img[y:y + h, x:x + w]
    #res.append(DetectorRes(crop, BBox(x, y, w, h)))
    res.append([x, crop])
  return sorted(res)


import os

path = '/content/drive/MyDrive/Handwritten_text_converter/Words/'
words = []
labels = []

for i in np.arange(1408, len(df), 1):
  img = cv2.imread(path_img + '/' + df['path'][i])
  if not os.path.exists(path + os.path.splitext(df['path'][i])[0]):
   os.makedirs(path + os.path.splitext(df['path'][i])[0])
  img = prepare_image(img, 150)
  res = detect(img, kernel_size = 25, sigma = 30, theta = 7, min_area = 300)
  tt = df['text'][i].split(" ")
  if len(res) == len(tt):
    for j in range(len(res)):
      path_name = os.path.splitext(df['path'][i])[0] + '/' + str(j) + '.tif'
      cv2.imwrite(path + path_name, res[j][1])
      words.append(path_name)
      labels.append(tt[j])


import pandas as pd

df = pd.DataFrame()
df['path'] = words
df['text'] = labels

df.to_csv('/content/drive/MyDrive/Handwritten_text_converter/words1.csv', index=False)